## First: Explore the data

*Assumptions:* Products: BARCODE (PK) ｜ User: ID (PK) ｜ Transaction: RECEIPT_ID (PK), USER_ID (FK), BARCODE(FK)

### 1. Data Validation Logic

To ensure comprehensive data quality validation, multiple perspectives need to be considered. While there are many possible approaches, based on the characteristics of these three datasets, I am focusing on **six key aspects**:

**Completeness** – Check for missing values in critical fields.

**Consistency** – Verify data formats and ensure uniform structure.

**Accuracy** – Ensure logical consistency, such as date relationships and value ranges.

**Uniqueness** – Identify duplicates in primary keys to prevent redundancy.

**Referential Integrity** – Validate foreign keys to ensure data relationships are maintained.

**Outliers** – Detect extreme values that may indicate data entry errors or anomalies.

### 2. Data Quality Issues

#### 2.1 Product Table | [Code](Data%20quality%20check_Product.ipynb)

**- Missing Values**

| Field         | Missing Values |
|--------------|---------------|
| CATEGORY_1   | 111           |
| CATEGORY_2   | 1,424         |
| CATEGORY_3   | 60,566        |
| CATEGORY_4   | 778,093       |
| MANUFACTURER | 226,474       |
| BRAND        | 226,472       |
| BARCODE      | 4,025         |

- CATEGORY_3 (60,000+ missing) and CATEGORY_4 is nearly 90% missing - can think about remove these columns.

- MANUFACTURER and BRAND have 226,000+ missing values, possibly causing classification issues.

**- Data Format** 
- BARCODE type is float64. Should be INT accroding to the ER diagram.

- Some barcodes' formats are incorrect like short barcodes (<8 digits) and long barcodes (>13 digits) 

![Barcode format](barcode_length.png)

**- Duplicates**
- BARCODE as primary key should not be unique and not null. Duplicate BARCODE Count is 4209 and null count is 4025.

#### 2.2 User Table | [Code](Data%20quality%20check_User.ipynb)

**- Missing Values**

| Field         | Missing Values |
|--------------|---------------|
| ID           | 0             |
| CREATED_DATE | 0             |
| BIRTH_DATE   | 3,675         |
| STATE        | 4,812         |
| LANGUAGE     | 30,508        |
| GENDER       | 5,892         |


**- Data Format**
- CREATED_DATE and BIRTH_DATE are stored as objects (strings) instead of datetime. Should change to datetime.

- GENDER has 11 unqiue values and some of them should be standardized to a single category like,
    - 'prefer_not_to_say', 'Prefer not to say', 'not_specified', 'unknown' 
    - 'not_listed' vs. "My gender
    - 'non_binary' and 'Non-Binary'. Convert all values to lowercase for consistency

#### 2.3 Transaction Table | [Code](Data%20quality%20check_Transaction.ipynb)

**- Missing Values**

| Field          | Missing Values |
|---------------|---------------|
| RECEIPT_ID    | 0             |
| PURCHASE_DATE | 0             |
| SCAN_DATE     | 0             |
| STORE_NAME    | 0             |
| USER_ID       | 0             |
| BARCODE       | 5,762         |
| FINAL_QUANTITY | 0            |
| FINAL_SALE    | 0             |

**- Date Format**
- Accroding to the ER Diagram, RECEIPT_ID URCHASE_DATE and SCAN_DATE should be DATETIME
- BARCODE type is float64. Should be INT accroding to the ER diagram.
- FINAL_SALE should be NUMERIC.

**- Duplicates**
- Each RECEIPT_ID appears twice, with the only difference being either:
    - FINAL_QUANTITY values (zero vs. any number ). "zero" should be changed to "0"
    - FINAL_SALE values (blank vs. any number). Blanks should be changed to "0"

**- Outliers**
- FINAL_SALE
    *Issue*: A few transactions have very high FINAL_SALE values (above 400), which is significantly larger than most other values.

- FINAL_QUANTITY
    *Issue*: There is a point where FINAL_QUANTITY exceeds 250, which is significantly higher than the majority of transactions.
    
![Quantity and Sales Distribution](final_quantity_vs_final_sale.png)


**- Logical Consistency**
- There are 94 records where PURCHASE_DATE is after SCAN_DATE, which is logically incorrect because a transaction cannot be scanned before it is purchased.

**- Referential Integrity**
- 17,603 transactions contain USER_IDs that do not exist in the user dataset.

- 11,028 transactions reference BARCODEs that do not exist in the product dataset.

### 3. Fields that are complex to interpret

The fields "FINAL_SALE" and "FINAL_QUANTITY" are challenging to interpret.

- FINAL_QUANTITY is expected to be an integer, but many values are decimals.
- FINAL_SALE has 480 transactions with a value of 0, possibly due to coupon usage.

----

## Second: provide SQL Queries

### Q1: What are the top 5 brands by receipts scanned among users 21 and over?


```SQL
SELECT p.BRAND, DISTINCT COUNT(t.RECEIPT_ID) AS total_receipts
FROM TRANSACTION_TAKEHOME t
JOIN USER_TAKEHOME u ON t.USER_ID = u.ID
JOIN PRODUCTS_TAKEHOME p ON t.BARCODE = p.BARCODE
WHERE DATE_PART('year', AGE(u.BIRTH_DATE)) >= 21 
    AND p.BRAND IS NOT NULL
    AND t.FINAL_QUANTITY != 'zero'
    AND t.FINAL_SALE IS NOT NULL
GROUP BY p.BRAND
ORDER BY total_receipts DESC
LIMIT 5;

### Q2: What are the top 5 brands by sales among users that have had their account for at least six months?

```SQL
SELECT p.BRAND, SUM(t.FINAL_SALE) AS total_sales
FROM TRANSACTION_TAKEHOME t
JOIN USER_TAKEHOME u ON t.USER_ID = u.ID
JOIN PRODUCTS_TAKEHOME p ON t.BARCODE = p.BARCODE
WHERE u.CREATED_DATE <= CURRENT_DATE - INTERVAL '6 months'
GROUP BY p.BRAND
ORDER BY total_sales DESC
LIMIT 5;

### Q3: Who are Fetch’s power users?

```sql
/**
Assumptions:
Who has the most receipts (Key Factor)
Who make the most purchases in the last 90 days (Second Factor)
Who spend the most - FINAL_SALE (Third Factor)
Who is active across multiple months (Fourth Factor)

Find the top 25 power users
**/

WITH user_activity AS (
    SELECT 
        t.USER_ID,
        COUNT(DISTINCT t.RECEIPT_ID) AS total_receipts,                         -- Total receipts scanned
        COUNT(CASE WHEN t.PURCHASE_DATE >= CURRENT_DATE - INTERVAL '90 days' THEN t.RECEIPT_ID END) AS purchases_last_90_days -- Purchases in last 90 days
        SUM(t.FINAL_SALE) AS total_spent,                                       -- Total amount spent
        COUNT(DISTINCT DATE_TRUNC('month', t.PURCHASE_DATE)) AS active_months,  -- Active months in transactions
    FROM TRANSACTION_TAKEHOME t
    JOIN USER_TAKEHOME u ON t.USER_ID = u.ID
    WHERE t.FINAL_SALE > 0                                                      -- Exclude invalid transactions
    GROUP BY t.USER_ID, u.CREATED_DATE
),
ranked_users AS (
    SELECT 
        USER_ID,
        total_receipts,
        total_spent,
        purchases_last_90_days,
        active_months,
        -- Weigh Score
        (total_receipts * 0.4) + (purchases_last_90_days * 0.3) + (total_spent * 0.2) + (active_months * 0.1) AS weighted_score,
        RANK() OVER (ORDER BY weighted_score DESC) AS user_rank
    FROM user_activity
)
SELECT * FROM ranked_users WHERE user_rank <= 25;


----

## Communicate with stakeholders

Hi Business Leader,

Following up on our last meeting, I have conducted a **data quality assessment** for the three tables based on the following aspects:
**Missing Values, Date Format, Duplicates, Outliers, Logical Consistency, and Referential Integrity**. During this assessment, I also noticed an interesting trend in the data.


*Quality Issues:*

The detailed findings are attached in three separate documents. Below, I would like to highlight key quality issues that require further discussion:

![Key Issues](key%20issues.png)


*Insight from the data:*

I made some assumptions here to identify a power user which should be:
Who has the most receipts
Who make the most purchases in the last 90 days
Who spend the most - FINAL_SALE
Who is active across multiple months

The results indicate that total spending heavily influences power user identification. There may be alternative approaches to defining power users, which we can discuss in our next meeting.

![Trends](Trend2.png)


I believe these data issues must be addressed promptly to ensure we accurately identify patterns and avoid missing critical insights. Resolving these issues will enhance data quality, leading to more reliable analysis that better serves our consumers and keeps us ahead of the curve.

Please let me know a suitable time to schedule a meeting and consult with other stakeholders who are familiar with the data logic, so we can align on the best course of action.

Best regards,

Yuting